In [157]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime


from dotenv import load_dotenv
import os
load_dotenv()


True

### Alcampo Web Scraping Script

In [ ]:
# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_alcampo ="https://www.alcampo.es/compra-online/alimentacion/c/WC10"
driver.get(url_alcampo)

#Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"cc_b_ok.auxOKButton.cookie-button")
cookies.send_keys(Keys.ENTER)

#Esperamos unos segundillos
time.sleep(3)

# Pinchamos en cualquier lado para quitar overlay z-score
expand_page = driver.find_element(By.CLASS_NAME,"ui-widget-overlay.first-time-overlay")
expand_page.click()


### Expandir Categorias

In [ ]:
category_open = driver.find_element(By.CSS_SELECTOR,"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria > div.facetHead.itemCerrado > h3 > a")
category_open.click()

time.sleep(3)

expand_category = driver.find_element(By.CSS_SELECTOR,"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria.coultarItemFacetPriceDesktop > div.facetValues > div.topFacetValues > span > a")
expand_category.click()

In [ ]:
# Recorrer cada categoria

for i in range(1,15):

    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,\
        f"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria.coultarItemFacetPriceDesktop > div.facetValues > div.allFacetValues > ul > li:nth-child({i}) > a")
    header_a_pinchar[0].click()
    time.sleep(5)
    print(header_a_pinchar[0].text.split("\n"))

### Alcampo Beautifulsoup script

In [155]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np



In [129]:
url ="https://www.alcampo.es/compra-online/alimentacion/c/WC10?q=%3Arelevance%3AcategoryChild%3AW16"



# hdr = {
#   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
#   'Accept-Encoding': 'gzip, deflate, br',
#   'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
#   'Connection': 'keep-alive',
#   'refere': url,
#   'cookie': cookie
# }


# response = requests.get(url,headers=hdr )

API_KEY = os.environ["SCARPE_OPS_API_KEY"]

response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params={
      'api_key': API_KEY,
      'url': url, 
  },
)

print('Body: ', response.content)






Body:  b'<!DOCTYPE html>\n<html lang="es">\n<head><link rel="preload" as="font" type="font/woff2" crossorigin href="/compra-online/_ui/desktop/theme-alcampo/css/fonts/fontawesome-webfont.woff2?v=4.3.0">\n<title>\r\n\t\tAlimentaci\xc3\xb3n | Alcampo Compra Online </title>\n\n<script src="/compra-online/_ui/shared/js/cookie-consent-3.1.0.js" data-cfasync="false" type="text/javascript" serverstartup="on"></script>\n<script type="cd84d1f77b949c6a23745e22-text/javascript" id="scriptCookies">\r\n        var labelCookies = {title: "Nosotros tambi\xc3\xa9n tenemos cookies\xe2\x80\xa6",\r\n            masinfo: "M\xc3\xa1s info",\r\n            privacidad: "pol\xc3\xadtica de privacidad",\r\n            politica: "pol\xc3\xadtica de cookies",\r\n            homep1: "El uso de cookies en Alcampo sirve para proporcionarte la mejor experiencia de compra posible en nuestra p\xc3\xa1gina web. Las cookies son b\xc3\xa1sicas para el funcionamiento \xc3\xb3ptimo de la p\xc3\xa1gina. Tambi\xc3\xa9n las u

In [130]:
html = response.content
soup = bs(html,"lxml")
soup

<!DOCTYPE html>
<html lang="es">
<head><link as="font" crossorigin="" href="/compra-online/_ui/desktop/theme-alcampo/css/fonts/fontawesome-webfont.woff2?v=4.3.0" rel="preload" type="font/woff2"/>
<title>
		Alimentación | Alcampo Compra Online </title>
<script data-cfasync="false" serverstartup="on" src="/compra-online/_ui/shared/js/cookie-consent-3.1.0.js" type="text/javascript"></script>
<script id="scriptCookies" type="cd84d1f77b949c6a23745e22-text/javascript">
        var labelCookies = {title: "Nosotros también tenemos cookies…",
            masinfo: "Más info",
            privacidad: "política de privacidad",
            politica: "política de cookies",
            homep1: "El uso de cookies en Alcampo sirve para proporcionarte la mejor experiencia de compra posible en nuestra página web. Las cookies son básicas para el funcionamiento óptimo de la página. También las usamos para realizar mejoras en la web o proponer contenidos personalizados. En definitiva tratamos de ofrecerte u

In [133]:
all_milk = soup.find_all("div",class_="productGridItemContainer")

In [154]:
name= []
images = []

for milk in all_milk:
    # print(milk.find("span").get_text(strip=True))
    name.append(milk.find("span").get_text(strip=True))
    images.append(milk.find("img").get_text(strip=True))
    print(milk.find("img").get_text(strip=True))
   
print(images)

















































['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
